In [3]:
!pip install torch transformers

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 721.4 kB/s eta 0:00:00m eta 0:00:010:00:03
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 5.4 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.2/63.2 MB 4.1 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 kB 1.8 MB/s eta 0:00:00 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 971.3 kB/s eta 0:00:00m eta 0:00:010:00:02
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 542.2 kB/s eta 0:00:00m eta 0:00:010:00:03
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 2.2 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.2/173.2 MB 1.9 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.1/177.1 MB 1.0 MB/s eta 0:00:00m eta 0:00:010:00:02
     ━━━━━━━━━━━━━━━━━

In [1]:
import torch
from transformers import AutoTokenizer, AutoModel
from helper_functions import *

# Load pre-trained AraBERT model and tokenizer
tokenizer = AutoTokenizer.from_pretrained('aubmindlab/bert-base-arabertv02')
model = AutoModel.from_pretrained('aubmindlab/bert-base-arabertv02')

FileNotFoundError: [Errno 2] No such file or directory: '/media/gharafi/Hamza-part/AINoLoPs/data/users_metadata'

In [2]:
rumors = pd.concat([pd.read_json(train_rumors_path), pd.read_json(dev_rumors_path)], ignore_index=True)

In [3]:
l = []

for i in rumors["tweet_text"]:
    
    tokens = tokenizer.tokenize(i)
    tokens = ['[CLS]'] + tokens + ['[SEP]']
    input_ids = tokenizer.convert_tokens_to_ids(tokens)
    input_ids = torch.tensor(input_ids).unsqueeze(0)

    
    # Get BERT embeddings for the input ids
    with torch.no_grad():
        outputs = model(input_ids)
        sentence_embedding = outputs[0][:, 0, :]  # take the final hidden state of the [CLS] token
    
    l.append(sentence_embedding)
    
X = np.vstack(l)

In [4]:
y = np.array(rumors["category"])
X.shape, y.shape

((150, 768), (150,))

In [5]:
rumors["category"].unique()

array(['politics', 'health', 'sports'], dtype=object)

In [6]:
X_train , y_train = X[:120] , y[:120]
X_test , y_test = X[120:] , y[120:]

X_train.shape , y_train.shape , X_test.shape , y_test.shape

((120, 768), (120,), (30, 768), (30,))

In [7]:
data_train = list(zip(X_train, y_train))

np.random.shuffle(data_train)

X_train, y_train = zip(*data_train)

X_train = np.array(X_train)
y_train = np.array(y_train)

X_train.shape , y_train.shape

((120, 768), (120,))

In [8]:
from ml_algo import *

In [9]:
rf_classifier(X_train, y_train, X_test, y_test)

Accuracy: 83.33%
Classification Report:
               precision    recall  f1-score   support

      health       0.75      0.90      0.82        10
    politics       1.00      0.70      0.82        10
      sports       0.82      0.90      0.86        10

    accuracy                           0.83        30
   macro avg       0.86      0.83      0.83        30
weighted avg       0.86      0.83      0.83        30



(0.8333333333333334,
 '              precision    recall  f1-score   support\n\n      health       0.75      0.90      0.82        10\n    politics       1.00      0.70      0.82        10\n      sports       0.82      0.90      0.86        10\n\n    accuracy                           0.83        30\n   macro avg       0.86      0.83      0.83        30\nweighted avg       0.86      0.83      0.83        30\n')

In [10]:
rf_classifier_hpt(X_train, y_train, X_test, y_test)

Best Hyperparameters:  {'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 50}
Best Mean Cross-validation Score:  0.8083333333333333
Test Accuracy:  0.8


({'max_depth': 10,
  'min_samples_leaf': 1,
  'min_samples_split': 2,
  'n_estimators': 50},
 0.8083333333333333,
 0.8)

In [11]:
lr_classifier(X_train, y_train, X_test, y_test)

Train Accuracy:  1.0
Test Accuracy:  0.8333333333333334


/home/gharafi/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


(1.0, 0.8333333333333334)

In [12]:
lr_classifier_hpt(X_train, y_train, X_test, y_test)

/home/gharafi/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/gharafi/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regressi

(0.1, 0.875, 0.8333333333333334)

In [13]:
svm_classifier(X_train, y_train, X_test, y_test)

0.8